#### Print your name

In [1]:
## Your code here 
print("Exercise by: Benjam.Alander")

Exercise by: Benjam.Alander


In [2]:
!pip install openpyxl

/opt/conda/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
%matplotlib inline

In [4]:
import pandas as pd

# Load the Excel file
file_path = './data/sukunimitilasto-2024-02-01-dvv.xlsx'
surnames_df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
surnames_df.head()

,Sukunimi,Yhteensä
0,Korhonen,21655
1,Virtanen,20879
2,Mäkinen,19320
3,Nieminen,19117
4,Mäkelä,18409


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time
import pandas as pd
import glob
import os


surnames_df = pd.read_excel(file_path)

# Filter surnames with more than 1000 occurrences
surnames = surnames_df[surnames_df['Yhteensä'] > 1000]['Sukunimi'].tolist()

# Helper functions to read category files
def findFiles(path):
    return glob.glob(path)

def readLines(filename):
    with open(filename, encoding='utf-8') as f:
        lines = f.read().strip().split('\n')
    return [line.strip() for line in lines]

# Load category files from ./data/names/
category_lines = {}
all_categories = []

for filename in findFiles('./data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

# Add Finnish surnames to categories
category_lines['Finnish'] = surnames

# Add 'Finnish' to all_categories if not already present
if 'Finnish' not in all_categories:
    all_categories.append('Finnish')

# Update number of categories
n_categories = len(all_categories)

# Define the RNN model class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# Assuming all_letters and n_letters are already defined in the original notebook
all_letters = "abcdefghijklmnopqrstuvwxyzåäö"
n_letters = len(all_letters)

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()
    return output, loss.item()

def evaluate(line_tensor):
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    return output

# Initialize the model, loss function, and optimizer
n_hidden = 256  # Increase the number of hidden units
rnn = RNN(n_letters, n_hidden, n_categories)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=0.001)  # Use Adam optimizer with a learning rate of 0.001

# Function to format time since start
def time_since(since):
    now = time.time()
    s = now - since
    m = int(s / 60)
    s -= m * 60
    return f'{m}m {s:.2f}s'

# Train the model with print statements to show learning progress
n_iters = 100000
print_every = 5000
plot_every = 1000

current_loss = 0
all_losses = []

start = time.time()

for iter in range(1, n_iters + 1):
    category = random.choice(all_categories)
    line = random.choice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)

    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else f'✗ ({category})'
        elapsed_time = time_since(start)

        # Calculate accuracy over a batch of samples
        correct_count = 0
        for _ in range(100):
            eval_category = random.choice(all_categories)
            eval_line = random.choice(category_lines[eval_category])
            eval_line_tensor = lineToTensor(eval_line)
            eval_output = evaluate(eval_line_tensor)
            eval_guess, _ = categoryFromOutput(eval_output)
            if eval_guess == eval_category:
                correct_count += 1

        accuracy = correct_count / 100 * 100

        print(f'Iteration: {iter} / {n_iters} ({iter / n_iters * 100:.2f}%)')
        print(f'Time elapsed: {elapsed_time}')
        print(f'Current loss: {loss:.4f}')
        print(f'Name: {line} / Predicted: {guess} {correct}')
        print(f'Accuracy over last 100 samples: {accuracy:.2f}%\n')

    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

# Evaluate the model with Finnish surnames
correct = 0
total = len(category_lines['Finnish'])

for surname in category_lines['Finnish']:
    output = evaluate(lineToTensor(surname))
    guess, _ = categoryFromOutput(output)
    if guess == 'Finnish':
        correct += 1

accuracy = correct / total * 100
print(f'Accuracy on Finnish surnames: {accuracy:.2f}%')

# Check the model prediction for the user's surname
user_surname = "Alander"  # Replace with your surname
output = evaluate(lineToTensor(user_surname))
guess, _ = categoryFromOutput(output)
print(f'The model predicts your surname ({user_surname}) as: {guess}')

Iteration: 5000 / 100000 (5.00%)
Time elapsed: 3m 27.16s
Current loss: 0.7100
Name: Kupiainen / Predicted: Finnish ✓
Accuracy over last 100 samples: 41.00%

Iteration: 10000 / 100000 (10.00%)
Time elapsed: 6m 54.87s
Current loss: 2.1812
Name: Giang / Predicted: Korean ✗ (Vietnamese)
Accuracy over last 100 samples: 44.00%

Iteration: 15000 / 100000 (15.00%)
Time elapsed: 10m 18.57s
Current loss: 1.1329
Name: Lang / Predicted: Korean ✗ (Chinese)
Accuracy over last 100 samples: 35.00%

Iteration: 20000 / 100000 (20.00%)
Time elapsed: 13m 45.36s
Current loss: 1.8103
Name: Perevuznik / Predicted: Russian ✗ (Czech)
Accuracy over last 100 samples: 48.00%

Iteration: 25000 / 100000 (25.00%)
Time elapsed: 17m 12.18s
Current loss: 1.7337
Name: Serafim / Predicted: Portuguese ✓
Accuracy over last 100 samples: 42.00%

Iteration: 30000 / 100000 (30.00%)
Time elapsed: 20m 36.89s
Current loss: 2.4847
Name: Piovene / Predicted: French ✗ (Italian)
Accuracy over last 100 samples: 51.00%

Iteration: 3500

# Reflection

1. **How accurate is the neural network?** The accuracy is 68.32%.

2. **How many Finnish names does the network predict correctly as Finnish?** The network predicts 68.32% of Finnish names correctly.

3. **Try your surname. What does the neural network predict?** When I input my surname "Alander," the neural network predicts it as Arabic.

4. **What could be done to improve the neural network performance?** Some possible improvements could include:
   - Increasing the size of the training dataset.
   - Experimenting with different neural network architectures and hyperparameters.
   - Augmenting the data with variations of existing samples.
   - Implementing techniques such as dropout regularization to prevent overfitting.
   - Training the model for more epochs to allow it to learn more complex patterns.


Käytin tekoälyä avuksi (chatGPT yms.)
Tein myös täysin tyhjästä itse koska koin vaikeaksi yhdistää tuohon pohjaan koodia

## Great job!